# Mono Camera Calibration

In [26]:
import cv2
import numpy as np
import os

# 1. 체스보드 패턴의 크기 (코너 수)
# [TODO] 체스보드 설정
chessboard_size =   # 내부 코너의 개수
square_size =   # 체스보드의 각 사각형 크기 (단위: mm)

# 2. 체스보드의 3D 좌표 생성 [Object_Point 3D]
objp = np.zeros((chessboard_size[0] * chessboard_size[1], 3), np.float32)
objp[:, :2] = np.mgrid[0:chessboard_size[0], 0:chessboard_size[1]].T.reshape(-1, 2)
objp *= square_size

# 3. 이미지에서 코너를 저장할 리스트
obj_points = []  # 3D 공간의 점
img_points = []  # 2D 이미지의 점

# 4. 이미지 파일 불러오기
num_camera = 1 # Cam2 저장할 때 변경 필수

image_folder = "data/images/cam" + str(num_camera)
image_files = [os.path.join(image_folder, f) for f in os.listdir(image_folder) if f.endswith(('.png', '.jpg', '.jpeg'))]

for frame in image_files:
    img = cv2.imread(frame)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # [TODO] Use cv2.findChessboardCorners() Functions
    ret, corners = 

    if ret:
        obj_points.append(objp)
        img_points.append(corners)

        cv2.drawChessboardCorners(img, chessboard_size, corners, ret)
        cv2.imshow('Corners', img)
        cv2.waitKey(100)

cv2.destroyAllWindows()

In [27]:
# [TODO] Use cv2.calibrateCamera() Functions
ret, camera_matrix, dist_coeffs, _, _ = 
dist_coeffs = dist_coeffs * 10**-13

print("Camera Matrix (Intrinsic Parameters):")
print(camera_matrix)

print("\nDistortion Coefficients:")
print(dist_coeffs)

Camera Matrix (Intrinsic Parameters):
[[1.12278714e+03 0.00000000e+00 7.85889866e+02]
 [0.00000000e+00 1.12187654e+03 4.59723025e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]

Distortion Coefficients:
[[-1.01361174e-14 -3.25954445e-15  3.04110498e-16 -3.24774974e-16
   5.16353152e-14]]


# Camera Matrix & Distortion Coefficients 저장

In [28]:
# Camera Matrix & Distiortion Coefficients txt 파일로 저장
cam1_matrix_path = "data/K_" + str(num_camera) +".txt"
cam1_distortion_path = "data/D_" + str(num_camera) +".txt"

np.savetxt(cam1_matrix_path, camera_matrix, fmt="%.8e", delimiter=" ")
np.savetxt(cam1_distortion_path, dist_coeffs[0][:2], fmt="%.8e", delimiter=" ")

# Save the pose vectors for all images.

In [29]:
pose_vectors_file = "data/all_pose_vectors_" + str(num_camera) + ".txt"
image_files.sort() # image 인덱스 순서로 정렬

with open(pose_vectors_file, "w") as f:
    for frame in image_files:
        img = cv2.imread(frame)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        ret, corners = cv2.findChessboardCorners(gray, chessboard_size, None)

        if ret:
            # [TODO] Use cv2.solvePnP() Functions
            ret, rvec, tvec = 

            rotation_matrix, _ = cv2.Rodrigues(rvec)

            angle = np.linalg.norm(rvec)
            axis = rvec.flatten() / angle if angle != 0 else rvec.flatten()
            wx, wy, wz = axis * angle

            # Translation vector (tx, ty, tz)
            tx, ty, tz = tvec.flatten()

            # Pose vector [wx, wy, wz, tx, ty, tz]
            pose_vector = [wx, wy, wz, tx, ty, tz]

            # Pose vector를 파일에 저장 (지정된 형식)
            f.write(" ".join(map(str, pose_vector)) + "\n")

# Cam2 도 동일하게 저장
### [K_2, D_2, all_pose_vectors_2]